In [ ]:
import pandas as pd
import itertools
#from tabulate import tabulate
import random
#from sklearn.metrics import euclidean_distances
import numpy as np

In [ ]:
#start point be 0, drop locations (1,2,3,4)
#items located at [(5,0),(2,5),(4,5),(6,5),(8,5)]
#D is the distance matrix
#n is number of orders
n=4
D=np.array([[0,13,11,17,11],[13,0,8,18,14],[11,8,0,11,6],[17,18,11,0,10],[11,14,6,10,0]])
np.fill_diagonal(D,9999)

In [ ]:
#define actions
actions=set(range(n+1))
#nodes to drop
nodes=actions.copy()
nodes.remove(0)

#define states
states=[]
for j in range(n+1):
  if j>0:
    sub=nodes.copy()
  if j==0:
    sub=actions.copy()
  sub.remove(j)
  for i in range(len(sub)+1):
    t=list(itertools.combinations(sub,i))
    for k in range(len(t)):
      states.append([j,t[k]])

#next states helper function gives the index of the next state in states list
transition=np.zeros([len(states),n])
transition.astype(int)
for i in range(len(states)):
  for j in range(n):
    new_set=set(states[i][1])
    new_set.discard(j)
    new_set=tuple(new_set)
    state=[j,new_set]
    print(states[i],state)
    transition[i,j]=states.index(state)


#reward function
rewards=np.zeros([len(states),n])
for i in range(len(states)):
  for j in range(n):
    bonus=0
    if(not(i==0) and(len(states[i][1])==0 and j==0)):
      bonus=1
    rewards[i,j]=-D[states[i][0],[j]]+bonus*100

#terminal states helper function
def is_terminal(state):
  if state==states.index([0,()]):
    return 1
  return 0

[0, ()] [0, ()]
[0, ()] [1, ()]
[0, ()] [2, ()]
[0, ()] [3, ()]
[0, (1,)] [0, (1,)]
[0, (1,)] [1, ()]
[0, (1,)] [2, (1,)]
[0, (1,)] [3, (1,)]
[0, (2,)] [0, (2,)]
[0, (2,)] [1, (2,)]
[0, (2,)] [2, ()]
[0, (2,)] [3, (2,)]
[0, (3,)] [0, (3,)]
[0, (3,)] [1, (3,)]
[0, (3,)] [2, (3,)]
[0, (3,)] [3, ()]
[0, (4,)] [0, (4,)]
[0, (4,)] [1, (4,)]
[0, (4,)] [2, (4,)]
[0, (4,)] [3, (4,)]
[0, (1, 2)] [0, (1, 2)]
[0, (1, 2)] [1, (2,)]
[0, (1, 2)] [2, (1,)]
[0, (1, 2)] [3, (1, 2)]
[0, (1, 3)] [0, (1, 3)]
[0, (1, 3)] [1, (3,)]
[0, (1, 3)] [2, (1, 3)]
[0, (1, 3)] [3, (1,)]
[0, (1, 4)] [0, (1, 4)]
[0, (1, 4)] [1, (4,)]
[0, (1, 4)] [2, (1, 4)]
[0, (1, 4)] [3, (1, 4)]
[0, (2, 3)] [0, (2, 3)]
[0, (2, 3)] [1, (2, 3)]
[0, (2, 3)] [2, (3,)]
[0, (2, 3)] [3, (2,)]
[0, (2, 4)] [0, (2, 4)]
[0, (2, 4)] [1, (2, 4)]
[0, (2, 4)] [2, (4,)]
[0, (2, 4)] [3, (2, 4)]
[0, (3, 4)] [0, (3, 4)]
[0, (3, 4)] [1, (3, 4)]
[0, (3, 4)] [2, (3, 4)]
[0, (3, 4)] [3, (4,)]
[0, (1, 2, 3)] [0, (1, 2, 3)]
[0, (1, 2, 3)] [1, (2, 3)]
[0, (1,

In [ ]:
states

[[0, ()],
 [0, (1,)],
 [0, (2,)],
 [0, (3,)],
 [0, (4,)],
 [0, (1, 2)],
 [0, (1, 3)],
 [0, (1, 4)],
 [0, (2, 3)],
 [0, (2, 4)],
 [0, (3, 4)],
 [0, (1, 2, 3)],
 [0, (1, 2, 4)],
 [0, (1, 3, 4)],
 [0, (2, 3, 4)],
 [0, (1, 2, 3, 4)],
 [1, ()],
 [1, (2,)],
 [1, (3,)],
 [1, (4,)],
 [1, (2, 3)],
 [1, (2, 4)],
 [1, (3, 4)],
 [1, (2, 3, 4)],
 [2, ()],
 [2, (1,)],
 [2, (3,)],
 [2, (4,)],
 [2, (1, 3)],
 [2, (1, 4)],
 [2, (3, 4)],
 [2, (1, 3, 4)],
 [3, ()],
 [3, (1,)],
 [3, (2,)],
 [3, (4,)],
 [3, (1, 2)],
 [3, (1, 4)],
 [3, (2, 4)],
 [3, (1, 2, 4)],
 [4, ()],
 [4, (1,)],
 [4, (2,)],
 [4, (3,)],
 [4, (1, 2)],
 [4, (1, 3)],
 [4, (2, 3)],
 [4, (1, 2, 3)]]

In [ ]:
#q-learning
q_values=np.ones([len(states),n])*1000

#parameters
alpha=0.2
gamma=0.9
epsilon=0.3

episodes=50
for i in range(episodes):
  initial_state=[0,tuple(nodes)]
  state=states.index(initial_state)
  done=False

  while not(done):
    r=np.random.random()
    if r>epsilon:
      action=np.argmax(q_values[state])
    else:
      action=np.random.randint(n)

    next_state=int(transition[state,action])

    reward=rewards[state,action]
    print(action,next_state,reward)
    if(is_terminal(state)):
      done=True

    td=reward+(gamma*np.max(q_values[next_state]))-q_values[state,action]
    new_q=(1-alpha)*(q_values[state,action])+(alpha*td)

    q_values[state,action]=new_q
    state=next_state

Streaming output truncated to the last 5000 lines.
1 19 -8.0
0 4 -13.0
2 27 -11.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
2 27 -9999.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
1 19 -9999.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
1 19 -9999.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
2 27 -9999.0
3 35 -11.0
2 27 -11.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
2 27 -9999.0
0 4 -11.0
2 27 -11.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
0 4 -11.0
2 27 -11.0
1 19 -8.0
3 35 -18.0
2 27 -11.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
0 4 -13.0
1 19 -13.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 -8.0
0 4 -11.0
2 27 -11.0
1 19 -8.0
2 27 -8.0
1 19 -8.0
2 27 

KeyboardInterrupt: ignored

In [ ]:
print(transition)

[[ 0. 16. 24. 32.]
 [ 1. 16. 25. 33.]
 [ 2. 17. 24. 34.]
 [ 3. 18. 26. 32.]
 [ 4. 19. 27. 35.]
 [ 5. 17. 25. 36.]
 [ 6. 18. 28. 33.]
 [ 7. 19. 29. 37.]
 [ 8. 20. 26. 34.]
 [ 9. 21. 27. 38.]
 [10. 22. 30. 35.]
 [11. 20. 28. 36.]
 [12. 21. 29. 39.]
 [13. 22. 31. 37.]
 [14. 23. 30. 38.]
 [15. 23. 31. 39.]
 [ 0. 16. 24. 32.]
 [ 2. 17. 24. 34.]
 [ 3. 18. 26. 32.]
 [ 4. 19. 27. 35.]
 [ 8. 20. 26. 34.]
 [ 9. 21. 27. 38.]
 [10. 22. 30. 35.]
 [14. 23. 30. 38.]
 [ 0. 16. 24. 32.]
 [ 1. 16. 25. 33.]
 [ 3. 18. 26. 32.]
 [ 4. 19. 27. 35.]
 [ 6. 18. 28. 33.]
 [ 7. 19. 29. 37.]
 [10. 22. 30. 35.]
 [13. 22. 31. 37.]
 [ 0. 16. 24. 32.]
 [ 1. 16. 25. 33.]
 [ 2. 17. 24. 34.]
 [ 4. 19. 27. 35.]
 [ 5. 17. 25. 36.]
 [ 7. 19. 29. 37.]
 [ 9. 21. 27. 38.]
 [12. 21. 29. 39.]
 [ 0. 16. 24. 32.]
 [ 1. 16. 25. 33.]
 [ 2. 17. 24. 34.]
 [ 3. 18. 26. 32.]
 [ 5. 17. 25. 36.]
 [ 6. 18. 28. 33.]
 [ 8. 20. 26. 34.]
 [11. 20. 28. 36.]]


In [ ]:
import pandas as pd
arr=[("1,2,[3,4],[5,6]]
pd.DataFrame(arr).to_csv("/content/output.csv", header=None, index=None)

In [ ]:
df=pd.read_csv("/content/possible_states_mixers.csv")
print(df)

         (0, 0)        (0, 1)        (0, 2)        (0, 3)        (1, 0)  \
0        (0, 0)        (0, 1)        (0, 2)        (0, 3)        (1, 0)   
1  (0, 0, 0, 0)  (0, 0, 0, 1)  (0, 0, 0, 2)  (0, 0, 0, 3)  (0, 0, 1, 0)   
2  (0, 0, 0, 0)  (0, 0, 0, 1)  (0, 0, 0, 2)  (0, 0, 0, 3)  (0, 0, 1, 0)   

         (1, 1)        (1, 2)        (1, 3)        (2, 0)        (2, 1)  ...  \
0        (1, 1)        (1, 2)        (1, 3)        (2, 0)        (2, 1)  ...   
1  (0, 0, 1, 1)  (0, 0, 1, 2)  (0, 0, 1, 3)  (0, 0, 2, 0)  (0, 0, 2, 1)  ...   
2  (0, 0, 1, 1)  (0, 0, 1, 2)  (0, 0, 1, 3)  (0, 0, 2, 0)  (0, 0, 2, 1)  ...   

   Unnamed: 246  Unnamed: 247  Unnamed: 248  Unnamed: 249  Unnamed: 250  \
0           NaN           NaN           NaN           NaN           NaN   
1  (3, 3, 1, 2)  (3, 3, 1, 3)  (3, 3, 2, 0)  (3, 3, 2, 1)  (3, 3, 2, 2)   
2  (3, 3, 1, 2)  (3, 3, 1, 3)  (3, 3, 2, 0)  (3, 3, 2, 1)  (3, 3, 2, 2)   

   Unnamed: 251  Unnamed: 252  Unnamed: 253  Unnamed: 254  Unnamed: 255  
0  